In [14]:
import sympy as sy
from sympy import pi, cos, sin, tan
from IPython.display import display

#q1, q2, q3, q4 = sy.symbols("q1, q2, q3, q4")  # 関節角度
t = sy.Symbol("t")
q1 = sy.Function("q1")
q2 = sy.Function("q2")
q3 = sy.Function("q3")
q4 = sy.Function("q4")
omega1 = sy.Function("omega1")
omega2 = sy.Function("omega2")
omega3 = sy.Function("omega3")
omega4 = sy.Function("omega4")

a1, a2, a3, a4 = sy.symbols("a1, a2, a3, a4")
b1, b2, b3, b4 = sy.symbols("b1, b2, b3, b4")
c1, c2, c3, c4 = sy.symbols("c1, c2, c3, c4")



l1, l2, l3, l4 = sy.symbols("l1, l2, l3, l4")  # リンク長さ
lg1, lg2, lg3, lg4 = sy.symbols("lg1, lg2, lg3, lg4")  # 重心までの長さ
m1, m2, m3, m4 = sy.symbols("m1, m2, m3, m4")  # 質量
Ig1, Ig2, Ig3, Ig4 = sy.symbols("I1, I2, I3, I4")  # 慣性モーメント
g = sy.Symbol("g")  # 重力加速度

def R(q):
    return sy.Matrix([
        [cos(q), -sin(q)],
        [sin(q), cos(q)],
    ])


def HTM(q, x, y):
    return sy.Matrix([
        [cos(q), -sin(q), x],
        [sin(q), cos(q), y],
        [0, 0, 1],
    ])


# 重心の位置
xg1 = R(q1(t)) * sy.Matrix([[lg1, 0]]).T
xg2 = R(q1(t)) * sy.Matrix([[l1, 0]]).T + \
    R(q1(t) + q2(t)) * sy.Matrix([[lg2, 0]]).T
xg3 = R(q1(t)) * sy.Matrix([[l1, 0]]).T +\
    R(q1(t) + q2(t)) * sy.Matrix([[l2, 0]]).T +\
        R(q1(t) + q2(t) + q3(t)) * sy.Matrix([[lg3, 0]]).T
xg4 = R(q1(t)) * sy.Matrix([[l1, 0]]).T +\
    R(q1(t) + q2(t)) * sy.Matrix([[l2, 0]]).T +\
        R(q1(t) + q2(t) + q3(t)) * sy.Matrix([[l3, 0]]).T +\
            R(q1(t) + q2(t) + q3(t) + q4(t)) * sy.Matrix([[lg4, 0]]).T

# 重心位置の時間微分
xg1_dot = sy.diff(xg1, t)
xg2_dot = sy.diff(xg2, t)
xg3_dot = sy.diff(xg3, t)
xg4_dot = sy.diff(xg4, t)


# 運動エネルギー
T1 = 1/2*m1*(xg1_dot[0,0]**2 + xg1_dot[1,0]**2) +\
    1/2*Ig1*sy.Derivative(q1(t),t)**2
T2 = 1/2*m2*(xg2_dot[0,0]**2 + xg2_dot[1,0]**2) +\
    1/2*Ig2*(sy.Derivative(q1(t),t) + sy.Derivative(q2(t),t))**2
T3 = 1/2*m3*(xg3_dot[0,0]**2 + xg3_dot[1,0]**2) +\
    1/2*Ig3*(sy.Derivative(q1(t),t) + sy.Derivative(q2(t),t) + sy.Derivative(q3(t),t))**2
T4 = 1/2*m4*(xg4_dot[0,0]**2 + xg4_dot[1,0]**2) +\
    1/2*Ig4*(sy.Derivative(q1(t),t) + sy.Derivative(q2(t),t) + sy.Derivative(q3(t),t) + sy.Derivative(q4(t),t))**2

# 位置エネルギー
U1 = m1*g*xg1[1,0]
U2 = m2*g*xg2[1,0]
U3 = m3*g*xg3[1,0]
U4 = m4*g*xg4[1,0]

# ラグランジアン
L = (T1 + T2 + T3 + T4) - (U1 + U2 + U3 + U4)

L = L.subs([
    (sy.Derivative(q1(t),t), omega1(t)),
    (sy.Derivative(q2(t),t), omega2(t)),
    (sy.Derivative(q3(t),t), omega3(t)),
    (sy.Derivative(q4(t),t), omega4(t)),
])

In [15]:
# dLdq1_dot = sy.diff(L, sy.Derivative(q1(t),t))
# dLdq2_dot = sy.diff(L, sy.Derivative(q2(t),t))
# dLdq3_dot = sy.diff(L, sy.Derivative(q3(t),t))
# dLdq4_dot = sy.diff(L, sy.Derivative(q4(t),t))

dLdq1_dot = sy.diff(L, omega1(t))
dLdq2_dot = sy.diff(L, omega2(t))
dLdq3_dot = sy.diff(L, omega3(t))
dLdq4_dot = sy.diff(L, omega4(t))

DdLdq1_dotDt = sy.diff(dLdq1_dot, t)
DdLdq2_dotDt = sy.diff(dLdq2_dot, t)
DdLdq3_dotDt = sy.diff(dLdq3_dot, t)
DdLdq4_dotDt = sy.diff(dLdq4_dot, t)

dLdq1 = sy.diff(L, q1(t))
dLdq2 = sy.diff(L, q2(t))
dLdq3 = sy.diff(L, q3(t))
dLdq4 = sy.diff(L, q4(t))

In [17]:
u1 = DdLdq1_dotDt - dLdq1
u2 = DdLdq2_dotDt - dLdq2
u3 = DdLdq3_dotDt - dLdq3
u4 = DdLdq4_dotDt - dLdq4

u = sy.Matrix([[u1, u2, u3, u4]]).T

u = u.subs([
    (sy.Derivative(omega1(t),t), c1),
    (sy.Derivative(omega2(t),t), c2),
    (sy.Derivative(omega3(t),t), c3),
    (sy.Derivative(omega4(t),t), c4),
])
u = u.subs([
    (sy.Derivative(q1(t),t), b1),
    (sy.Derivative(q2(t),t), b2),
    (sy.Derivative(q3(t),t), b3),
    (sy.Derivative(q4(t),t), b4),
])
u = u.subs([
    (q1(t), a1),
    (q2(t), a2),
    (q3(t), a3),
    (q4(t), a4),
])
u = u.subs([
    (omega1(t), b1),
    (omega2(t), b2),
    (omega3(t), b3),
    (omega4(t), b4),
])

print(u[0,0])
_u = sy.expand(u[0,0]).coeff(c1, 1)

print(_u)


1.0*I1*c1 + 0.5*I2*(2*c1 + 2*c2) + 0.5*I3*(2*c1 + 2*c2 + 2*c3) + 0.5*I4*(2*c1 + 2*c2 + 2*c3 + 2*c4) + g*lg1*m1*cos(a1) + g*m2*(l1*cos(a1) + lg2*cos(a1 + a2)) + g*m3*(l1*cos(a1) + l2*cos(a1 + a2) + lg3*cos(a1 + a2 + a3)) + g*m4*(l1*cos(a1) + l2*cos(a1 + a2) + l3*cos(a1 + a2 + a3) + lg4*cos(a1 + a2 + a3 + a4)) + 0.5*m1*(2*c1*lg1**2*sin(a1)**2 + 2*c1*lg1**2*cos(a1)**2) - 0.5*m2*((-2*b1*l1*sin(a1) - 2*lg2*(b1 + b2)*sin(a1 + a2))*(b1*l1*cos(a1) + lg2*(b1 + b2)*cos(a1 + a2)) + (-b1*l1*sin(a1) - lg2*(b1 + b2)*sin(a1 + a2))*(-2*b1*l1*cos(a1) - 2*lg2*(b1 + b2)*cos(a1 + a2))) + 0.5*m2*((-2*l1*sin(a1) - 2*lg2*sin(a1 + a2))*(-b1**2*l1*cos(a1) - c1*l1*sin(a1) - lg2*(b1 + b2)**2*cos(a1 + a2) - lg2*(c1 + c2)*sin(a1 + a2)) + (2*l1*cos(a1) + 2*lg2*cos(a1 + a2))*(-b1**2*l1*sin(a1) + c1*l1*cos(a1) - lg2*(b1 + b2)**2*sin(a1 + a2) + lg2*(c1 + c2)*cos(a1 + a2)) + (-2*b1*l1*sin(a1) - 2*lg2*(b1 + b2)*sin(a1 + a2))*(b1*l1*cos(a1) + lg2*(b1 + b2)*cos(a1 + a2)) + (-b1*l1*sin(a1) - lg2*(b1 + b2)*sin(a1 + a2))*(-2

In [ ]:
f = open('sice_dynamics.txt', 'w')
for i, j in enumerate(u):
    s = '\nu' + str(i) + '='
    f.write(s)
    f.write(str(j))
f.close()

In [9]:
tau1, tau2, tau3, tau4 = sy.symbols("tau1, tau2, tau3, tau4")
tau = sy.Matrix([[tau1, tau2, tau3, tau4]]).T

f = tau - u

sol = sy.solve(
    [f[0,0], f[1,0], f[2,0], f[3,0]],
    [c1, c2, c3, c4]
)

KeyboardInterrupt: 

In [ ]:
sol

Error: Session cannot generate requests

In [ ]:
print(u)


Matrix([[1.0*I1*c1 + 0.5*I2*(2*c1 + 2*c2) + 0.5*I3*(2*c1 + 2*c2 + 2*c3) + 0.5*I4*(2*c1 + 2*c2 + 2*c3 + 2*c4) + g*lg1*m1*cos(a1) + g*m2*(l1*cos(a1) + lg2*cos(a1 + a2)) + g*m3*(l1*cos(a1) + l2*cos(a1 + a2) + lg3*cos(a1 + a2 + a3)) + g*m4*(l1*cos(a1) + l2*cos(a1 + a2) + l3*cos(a1 + a2 + a3) + lg4*cos(a1 + a2 + a3 + a4)) + 0.5*m1*(2*c1*lg1**2*sin(a1)**2 + 2*c1*lg1**2*cos(a1)**2) - 0.5*m2*((-2*b1*l1*sin(a1) - 2*lg2*(b1 + b2)*sin(a1 + a2))*(b1*l1*cos(a1) + lg2*(b1 + b2)*cos(a1 + a2)) + (-b1*l1*sin(a1) - lg2*(b1 + b2)*sin(a1 + a2))*(-2*b1*l1*cos(a1) - 2*lg2*(b1 + b2)*cos(a1 + a2))) + 0.5*m2*((-2*l1*sin(a1) - 2*lg2*sin(a1 + a2))*(-b1**2*l1*cos(a1) - c1*l1*sin(a1) - lg2*(b1 + b2)**2*cos(a1 + a2) - lg2*(c1 + c2)*sin(a1 + a2)) + (2*l1*cos(a1) + 2*lg2*cos(a1 + a2))*(-b1**2*l1*sin(a1) + c1*l1*cos(a1) - lg2*(b1 + b2)**2*sin(a1 + a2) + lg2*(c1 + c2)*cos(a1 + a2)) + (-2*b1*l1*sin(a1) - 2*lg2*(b1 + b2)*sin(a1 + a2))*(b1*l1*cos(a1) + lg2*(b1 + b2)*cos(a1 + a2)) + (-b1*l1*sin(a1) - lg2*(b1 + b2)*sin(a1 +